# Notebook Activity A1.8 Testing for fixed effects in a wage equation

This is the second of three notebooks where you will be practicing the use of estimators that are available for panel data. This notebook will cover two fixed effects estimators, the least squares dummy variables (LSDV) estimator and the within groups (WG) estimator. You will also be testing whether fixed effects are needed compared to pooled OLS.

In this notebook you will make full use of the panel dataset `psid` to estimate a Mincerian equation. Recall the data in `psid` corresponds to a balanced panel on 595 individuals over 7 time periods. As a reminder, the variables are described as follows.

* `exper`: the number of years of full-time experience
* `occupation`: taking values 1 (blue-collar occupation) and 0 (other occupations)
* `gender`: taking values 0 (male) and 1 (female)
* `educ`: the number of years of education
* `ethnicity`: taking values 1 (black) and 0 (otherwise)
* `wageLog`: natural logarithm of hourly wages (in cents of a dollar)
* `period`: year of the study that the observation relates to
* `id`: individual.


## (a) Getting started

### (i)

Start by running the following code.

In [ ]:
# Load the M348 package
library(M348)
# Load the psid data frame
data(psid)
# Check the data frame has loaded OK
head(psid, 14)
tail(psid, 14)

###  <a class="tocSkip">

The Mincer equation we will estimate in this notebook is

\begin{equation*}\texttt{wageLog}_{it} = \beta_0 +f_i +  \beta_1 \texttt{educ}_{i} + \beta_2 \texttt{exper}_{it} +  \beta_3\texttt{exper}^2_{it} + +  \beta_4\texttt{occupation}_{it}+ \beta_5 \texttt{gender}_{i} + \beta_6 \texttt{ethnicity}_{i}  +  u_{it}\end{equation*} 

where the $f_i$ are fixed effects and $u_{it}$ is a well-behaved error term.

### (ii)

Create a new variable `experSq` the values of which are the squares of the values in `exper` and add it to the `psid` data frame.

#### Solution <a class="tocSkip">

The required code is given below. Note that running this code does not produce any output.

In [2]:
psid$experSq <- psid$exper^2

## (b) Checking for time invariance

You may recall from Section 5.2.2 that the Within-groups (WG) estimator removes fixed effects from the regression by demeaning all model variables. In doing so however, it also removes all time invariant regressors from the transformed regression (see Table 15, Subsection 5.2.2 where `educ` was time invariant for all individuals in the data and its coefficient could not be identified using WG estimation). So one important stage of fixed effects estimation, and of descriptive analysis of panel data in general, is to confirm which regressors are time invariant at the individual level. That is, whether for each individual the value of the regressor is the same for all time periods. 

To do this, we will make use of the following command

`by(X, individual, function)`

to calculate a summary statistic of a variable, for each individual in the panel data frame.

In this function
* `X` is the variable we are interested in.
* `individual` is the variable that indicates which individual an observation corresponds to
* `function` is the name of the function that gives the summary statistic. (For example `function` might be `mean` or `sd` or `var`.)

So for example `by(psid$educ, psid$id, psid$sd)` calculates the standard deviation of `educ` in `psid` for each individual.

### (i)

If `educ` is time invariant, what would be the standard deviation of it for each individual? What can we say about the the standard deviation of `educ` for each individual if `educ` is not time invariant? 

#### Solution <a class="tocSkip">

If `educ` is time invariant then every individual will have the same value for every time point. So the standard deviation of these values must be zero.

If `educ` is not time invariant for each individual then, at least one of the values will be different from the values of the other periods. So for this individual, the standard deviation will be greater than zero. The maximum value of the standard deviations calculated will be larger than zero.

### (ii)

By running the code below, calculate the maximum of the individual standard deviations for `educ`. Why does the result indicate that `educ` is time invariant?

In [ ]:
max(by(psid$educ, psid$id, sd))

#### Solution <a class="tocSkip">

The maximum individual standard deviation, which means that all the individual standard deviations must be 0. (Remember that a standard deviation cannot be less than 0!) This means that for every individual the value of `educ` does not vary. Hence `educ` is time invariant. 

### (iii)  

Use the empty code cell below to confirm that the variables `gender` and `ethnicity` are also time invariant. (And that the variables `exper`, `occupation` and `wageLog` do vary with time, for at least one individual.)

#### Solution <a class="tocSkip">

The required code and resulting output are given below.

The variables `gender` and `ethnicity` are all time invariant because for any single value of `id` (i.e. the data for an individual) the standard deviation is 0. 

The standard deviation for `exper`, `occupation` and `wageLog` is greater than 0 for at least one individual, indicating that the values must vary for at least one individual. 

In [4]:
max(by(psid$gender,psid$id, sd))
max(by(psid$ethnicity, psid$id, sd))
max(by(psid$exper, psid$id, sd))
max(by(psid$occupation,psid$id, sd))
max(by(psid$wageLog, psid$id, sd))

[1] 0

[1] 0

[1] 2.160247

[1] 0.5345225

[1] 0.9338521

## (c) Using the LSDV estimator

Recall from Box 25 in Subsection 5.2.1 that an estimable version of the fixed effects model can be written as 

$ Y_{it} = \beta_0 + \alpha_2 D_2 + · · · + \alpha_N D_N + \beta_1 X_{1it} + \cdots + \beta_K X_{Kit} + u_{it},$

where $D_i$ is a dummy variable for individual $i$. That is, it takes the value 1 when an observation corresponds to individual $i$ and 0 otherwise.

We could calculate all these dummy variables explicitly in R. However, we don't have to, just as we didn't when fitting linear models that have categorical variables as explanatory variables in Units 3 and 4. Instead all we need to do is tell R to treat the variable representing individuals (`id` in the case of `psid`) as a factor. We then include that factor in our model specification. 

So the LSDV estimator is used when we give commands of the form

`lsdvModel <- lm(Y ~ x1 + x2 + ... xK + I, data = dataFrame)`,

where `I` is a factor that corresponds to the cross-sectional units of the panel data set (and assuming the vectors `Y`, `x1`, `x2`, ..., `xK` and `I` are in a data frame called `dataFrame`).



### (i)

Use the code cell below to convert `id` to a factor. (Look back at Notebook activity 1.7 if you need a reminder of how to do this.)

#### Solution <a class="tocSkip">

The required code is given below. Note that running this code will not produce any output. 

In [5]:
psid$id <- factor(psid$id)

### (ii)

Using the code cell below, obtain LSDV estimates for this Mincer equation based on the `psid` dataset. Store the results in an object called `lsdvModel` and obtain a summary of the output.

#### Solution <a class="tocSkip">

The required code and resulting output are given below.

In [6]:
# Apply the LSDV estimator
lsdvModel <- lm(wageLog ~ educ + exper + experSq + occupation + gender + 
                ethnicity + id, data = psid)
# Display the output
summary(lsdvModel)



Call:
lm(formula = wageLog ~ educ + exper + experSq + occupation + 
    gender + ethnicity + id, data = psid)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.81190 -0.05091  0.00437  0.06090  1.94304 

Coefficients: (3 not defined because of singularities)
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  4.315e+00  2.907e-01  14.842  < 2e-16 ***
educ         1.093e-01  2.762e-02   3.958 7.72e-05 ***
exper        1.138e-01  2.468e-03  46.133  < 2e-16 ***
experSq     -4.275e-04  5.454e-05  -7.838 6.00e-15 ***
occupation  -1.731e-02  1.364e-02  -1.269 0.204462    
gender      -1.070e-02  8.332e-02  -0.128 0.897817    
ethnicity   -1.059e+00  1.005e-01 -10.538  < 2e-16 ***
id2         -2.291e+00  7.753e-02 -29.552  < 2e-16 ***
id3         -8.695e-02  8.138e-02  -1.068 0.285381    
id4         -1.308e+00  1.124e-01 -11.639  < 2e-16 ***
id5         -5.556e-01  1.668e-01  -3.331 0.000875 ***
id6         -1.457e+00  8.440e-02 -17.259  < 2e-16 ***
id7         -1

###  <a class="tocSkip">

Notice in the (long!) output you obtained in part (iii), a value is given for `id2` to `id592`. Each of these values, in combination with the intercept, give a different intercept for each individual. Which is how individual effects are modelled and estimated.

However, as there are 595 individuals in this panel dataset, it means four individual dummy coefficients have not been estimated. That the value for the first individual (`id1`) is missing should not surprise you. As you have seen with when other models that include a factor, the first level is dropped when the model already has an intercept applied to all observations. The lack of an estimate for the last three individuals (`id593`, `id594` and `id595`) is a result of `educ`, `gender` and `ethnicity` being time invariant in this panel. `R` chooses to drop a dummy variable for each time invariant regressor. It has not been possible to estimate the coefficient of `educ`, `gender` and `ethnicity` *and* all coefficients for individuals `id2` to `id595`. 

Now we could estimate the coefficients for `id593`, `id594` and `id595` by not estimating three of the other dummy variables but, as in Example 6 in Subsection 5.2.1, we might get a very different estimate for `educ`, `gender` and `ethnicity`!

### (iii)

Using the code cell below, obtain the LSDV estimates for the Mincer equation which only includes `exper`, `experSq` and `occupation`. Are all dummy variable coefficients except the first one (`id1`) estimated?

#### Solution <a class="tocSkip">

The required code and resulting output are given below. Yes now all dummy variable coefficients have been estimated (except for the first one since this model has an intercept and that would cause the dummy variable trap).

In [7]:
lsdvReducedModel <- lm(wageLog ~ exper + experSq + occupation + id, data = psid)
summary(lsdvReducedModel)


Call:
lm(formula = wageLog ~ exper + experSq + occupation + id, data = psid)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.81190 -0.05091  0.00437  0.06090  1.94304 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  5.299e+00  5.897e-02  89.863  < 2e-16 ***
exper        1.138e-01  2.468e-03  46.133  < 2e-16 ***
experSq     -4.275e-04  5.454e-05  -7.838 6.00e-15 ***
occupation  -1.731e-02  1.364e-02  -1.269 0.204462    
id2         -2.073e+00  8.867e-02 -23.375  < 2e-16 ***
id3          2.409e-01  8.272e-02   2.913 0.003603 ** 
id4         -2.269e+00  8.909e-02 -25.470  < 2e-16 ***
id5          2.095e-01  8.259e-02   2.536 0.011244 *  
id6         -1.129e+00  8.727e-02 -12.934  < 2e-16 ***
id7         -8.795e-01  8.448e-02 -10.411  < 2e-16 ***
id8         -1.359e+00  8.642e-02 -15.723  < 2e-16 ***
id9          9.324e-01  8.136e-02  11.461  < 2e-16 ***
id10         5.474e-01  8.136e-02   6.728 2.00e-11 ***
id11        -1.112e+00  8.669e-02

## (d) Using the WG estimator

Recall from Box 26 in Subsection 5.2 that the within groups (WG) estimator applies OLS to models of the form

$$ Y_{it}^∗ = \beta_1 X_{1it}^* + \cdots + \beta_K X_{Kit}^∗ + u_{it},$$

where for a variable $Z_{it}$, $Z_{it}^∗ = (Z_{it} − \overline{Z}_i)$. That is, $Z_{it}^∗$ is a demeaned variable, where to each observation the mean subtracted is its individual level mean.

One way of doing this demeaning, which works for balanced panel data structured in a data frame in the same way that `psid` has been (i.e. all the values for an individual sorted by year in consecutive rows, and all individuals with the same number of years), is to first use a command of the form
    
`zMeans <- rep(by(z, i, mean), each=T)`
  
This command calculates the mean of `z` separately for each level in the variable `i`, and creates a vector where each value is repeated $T$ times. Total length of the vector is the number of levels ($N$) times the number of repetitions ($T$). Then
    
`zStar <- z - zMeans`

creates a variable `zStar` which contains the demeaned values of `z`.


### (i)

By adding to the code below, obtain demeaned versions of `wageLog`, `exper`,  `experSq` and `occupation` (called `wageLogStar`, `experStar`, `experSqStar` and `occupationStar` respectively).

In [ ]:
wageLogMeans <- rep(by(psid$wageLog, psid$id, mean), each = 7)
wageLogStar <- psid$wageLog - wageLogMeans
summary(wageLogStar)





#### Solution <a class="tocSkip">

The required code is given below. Note that running this code will not produce any output.

In [9]:
wageLogMeans <- rep(by(psid$wageLog, psid$id, mean), each = 7)
wageLogStar <- psid$wageLog - wageLogMeans
experMeans <- rep(by(psid$exper, psid$id, mean), each = 7)
experStar <- psid$exper - experMeans
experSqMeans <- rep(by(psid$experSq, psid$id, mean), each = 7)
experSqStar <- psid$experSq - experSqMeans
occupationMeans <- rep(by(psid$occupation, psid$id, mean), each = 7)
occupationStar <- psid$occupation - occupationMeans

### (ii)

Combine the variables `wageLogStar`, `experStar`, `experSqStar` and `occupationStar` into a data frame called `psidStar`. Obtain summaries of these variables.

#### Solution <a class="tocSkip">

The required code and resulting output are given below.

In [10]:
psidStar <- data.frame(wageLogStar, experStar, experSqStar, occupationStar)
summary(psidStar)

  wageLogStar          experStar   experSqStar   occupationStar   
 Min.   :-1.894538   Min.   :-3   Min.   :-283   Min.   :-0.8571  
 1st Qu.:-0.160712   1st Qu.:-2   1st Qu.: -53   1st Qu.: 0.0000  
 Median : 0.007657   Median : 0   Median :  -4   Median : 0.0000  
 Mean   : 0.000000   Mean   : 0   Mean   :   0   Mean   : 0.0000  
 3rd Qu.: 0.172137   3rd Qu.: 2   3rd Qu.:  53   3rd Qu.: 0.0000  
 Max.   : 1.944746   Max.   : 3   Max.   : 293   Max.   : 0.8571  

### (iii)

By looking at the results you obtained in part (ii), does the demeaning seem to have worked?

#### Solution <a class="tocSkip">

Yes, as far as it is possible to tell. As the values for each individual should be 0 after the demeaning, then the overall mean should also be 0. This is indeed the case. 

### (iv)

Remove the `#`, adapt the code to apply the OLS estimator to the demeaned data and obtain a summary of the results

In [11]:
# wgModel <- lm(yStar ~ x1Star + ... + xKStar, data = dataFrame)
# summary(wgModel)

#### Solution <a class="tocSkip">

The required code and resulting output is as follows.

In [12]:
wgModel <- lm(wageLogStar ~ experStar + experSqStar + occupationStar, 
              data = psidStar)
summary(wgModel)


Call:
lm(formula = wageLogStar ~ experStar + experSqStar + occupationStar, 
    data = psidStar)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.81190 -0.05091  0.00437  0.06090  1.94304 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)    -8.036e-18  2.184e-03   0.000    1.000    
experStar       1.138e-01  2.285e-03  49.826   <2e-16 ***
experSqStar    -4.275e-04  5.049e-05  -8.465   <2e-16 ***
occupationStar -1.731e-02  1.263e-02  -1.371    0.171    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.1409 on 4161 degrees of freedom
Multiple R-squared:  0.6566,	Adjusted R-squared:  0.6564 
F-statistic:  2652 on 3 and 4161 DF,  p-value: < 2.2e-16


### (v)

Compare the results you obtained in part (d)(iv) with those you obtained in part (c)(ii).

#### Solution <a class="tocSkip">

The first thing you probably noticed is that the output from the WG esimator is much shorter. The WG does not estimate values for dummy variables - almost 600 of them!

Also the estimates for `exper`, `experSq` and `occupation` are identical. However the standard errors are not the same. This is because the demeaned data are not purely original data. They contain means of the data, which changes the assumptions we make on the standard errors of our data.

## (e) Using the plm package to implement the  WG estimator

In Notebook activity A1.7 you saw how the `plm()` function within plm package can be used to estimate models using the pooled estimator. In this part you will learn how to use this function to estimate models using WG fixed effect estimator. 

You will see that using `plm()` the demeaning of the variables is done for you, and it will automatically handle time-invariant variables appropriately so that you don't have to remove them from the model specification first. More importantly it also provides the correct standard errors for all the terms in the model. 

Before the `plm()` function is used to estimate a model using the WG estimator, we need to create a panel data frame that contains all the variables in the model we want to estimate.

### (i)

Use the code cell below to load the plm package and then create a panel data frame called `psidPanel` from `psid`. (Look back at Notebook activity A1.5 if you need a reminder of how to create a panel data frame.)

#### Solution <a class="tocSkip">

The required code is given below. Note that running this code will not produce any output.

In [13]:
library(plm)
psidPanel <- pdata.frame(psid,index = c("id", "period"))

###  <a class="tocSkip">

To estimate a WG  estimator using `plm()`, the basic command is:

`panelWGReg <- plm(Y ~ x1 + x2 + ... +  xk, data = panelDataFrame, effect = "individual", model = "within")`.

Notice that the only difference between this and the command given in Notebook activity A1.7 to perform pooled OLS is that the argument `model = "pooling"` has changed to `model = "within"`.

### (ii)

Remove the `#` and adapt the code below, to estimate the Mincerian equation specified in part (c) using a within groups estimates, this time using the `plm()` and storing the results in `mincerWGUsingPlm`. Also obtain a summary of the results. 

In [14]:
# panelWGReg <- plm(Y ~ x1 + x2 + ... +  xK, 
#        data = panelDataFrame, effect = "individual", model = "within")
# summary(panelWGReg)

#### Solution <a class="tocSkip">

The required code and resulting output are as follows. 

In [15]:
mincerWGUsingPlm <- plm(wageLog ~ educ + exper + experSq + occupation + 
                        gender + ethnicity, data = psidPanel, 
                        effect = "individual", model = "within")
summary(mincerWGUsingPlm)

Oneway (individual) effect Within Model

Call:
plm(formula = wageLog ~ educ + exper + experSq + occupation + 
    gender + ethnicity, data = psidPanel, effect = "individual", 
    model = "within")

Balanced Panel: n = 595, T = 7, N = 4165

Residuals:
      Min.    1st Qu.     Median    3rd Qu.       Max. 
-1.8119031 -0.0509124  0.0043674  0.0609016  1.9430359 

Coefficients:
              Estimate  Std. Error t-value  Pr(>|t|)    
exper       1.1384e-01  2.4676e-03 46.1330 < 2.2e-16 ***
experSq    -4.2745e-04  5.4537e-05 -7.8379 6.002e-15 ***
occupation -1.7312e-02  1.3641e-02 -1.2692    0.2045    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    240.65
Residual Sum of Squares: 82.64
R-Squared:      0.6566
Adj. R-Squared: 0.59912
F-statistic: 2273.41 on 3 and 3567 DF, p-value: < 2.22e-16

### (iii)

Compare the results you obtained in part (e)(ii) with those in part (d)(iv).

#### Solution <a class="tocSkip">

As expected, the estimates in part (e)(ii) are the same as in part (d)(iv), but the standard errors are larger.

## (f) Are fixed effects needed?

In Notebook activity A1.7 we estimated a pooled OLS whereas in this activity we have been using fixed effects estimators. It is possible to test whether a pooled model is sufficient, by using an $F$-test in the same way as you have done in previous units because the following models are nested models. 

* Pooled: `wageLog ~ educ + exper + exper^2 + occupation + gender + ethnicity`
* Fixed: `wageLog ~ educ + exper + exper^2 + occupation gender + ethnicity + id`




### (i) 

Estimate the Mincer equation 

$$\texttt{wageLog}_{it} = \beta_0 + \beta_1 \texttt{educ}_{i} + \beta_2 \texttt{exper}_{it} +  \beta_3\texttt{exper}^2_{it} +   +  \beta_4\texttt{occupation}_{it}+ \beta_5 \texttt{gender}_{i} + \beta_6 \texttt{ethnicity}_{it} + \nu_{it}.$$ 
using a pooled estimator and the `lm()` function, storing the results in `pooledModel`.

#### Solution <a class="tocSkip">

The required code is given below. Note that this will not produce any output.

In [16]:
pooledModel <- lm(wageLog ~ educ + exper + experSq + occupation + gender + 
                  ethnicity, data = psid)

### (ii)

By running the following code, test whether the pooled model is sufficient.

In [ ]:
anova(pooledModel,lsdvModel)

#### Solution <a class="tocSkip">

The $p$-value for this test, given in the `Pr(>F)` column of the second row, is very small, which indicates that the smaller model, the pooled model, in this case, is not sufficient. Thus we should prefer the fixed effects estimator over the pooled OLS estimator.

###  <a class="tocSkip">

The code in part (ii) works because both models have been estimated using `lm()`. Using `anova()` does not work when the models are estimated using `plm()`. However all is not lost, there is a command `pFtest()` that works instead. First we need to fit the pooled model using `plm()`.

### (iii)

Estimate the Mincer equation 
$$\texttt{wageLog}_{it} = \beta_0 + \beta_1 \texttt{educ}_{i} + \beta_2 \texttt{exper}_{it} +  \beta_3\texttt{exper}^2_{it} +   + \beta_4 \texttt{gender}_{i} + \beta_5 \texttt{ethnicity}_{it} +  \beta_6\texttt{occupation}_{it}+ \nu_{it}.$$ 
using a pooled estimator and the `plm()` function, storing the results in `mincerPooledUsingPlm`.

#### Solution <a class="tocSkip">

The required code is as follows. Note that this will not produce any output.

In [18]:
mincerPooledUsingPlm <- plm(wageLog ~ educ + exper + experSq + occupation + 
                            gender + ethnicity, data = psidPanel, 
                            model = "pooling")

### (iv)

By running the following code, repeat the test of whether the pooled model is sufficient, this time using output produced by `plm()`. 

In [ ]:
pFtest(mincerWGUsingPlm, mincerPooledUsingPlm)

Note that in part (iv) the command uses the WG model and not LSDV. This is because the WG is less sensitive and unstable.

### (v)

Compare the results you obtained in part (iv) with those you obtained in part (ii).

#### Solution <a class="tocSkip">

We can see that, although formatted differently the results obtained in part (iv) are essentially the same as those obtained in part (ii).